In [4]:
import rockhopper
from rockhopper import loadPLY, exportZA
import numpy as np
import os

#### Loading data

Lets load an example point cloud. `rockhopper` natively supports PLY format point clouds, which can easily be created/converted using e.g., [CloudCompare](https://www.danielgm.net/cc/).  

In the following we load an example point cloud (you will probably need to change the path) and extract the point coordinates, colours, normals and scalar attributes.

In [5]:
# load data from a PLY file
cloud = loadPLY('/Users/thiele67/Documents/data/dolomites/HySpex/dolomites_5cm_UTM33N_clipped.ply')

# retrieve attributes from resulting dict
xyz = cloud['xyz']
rgb = cloud['rgb']
normals = cloud['normals']
attr = cloud['attr']
attr_names =  cloud['names']

print("Loaded %d points containing %d attributes (%s)"%(len(xyz), len(attr_names), attr_names))

Loaded 16224567 points containing 1 attributes (['scalar_Illuminance_(PCV)'])


#### Preparing for Zarr

Point data is served to our virtual field trip as a chunked array, using a compression and streaming tool called [zarr](https://zarr.readthedocs.io/en/stable/). 

To prepare for this, we need to concatenate our attributes into a single (n,d) array. This will contain all our (n) point coordinates, colours and attributes (d). At a minimum, coordinates and colours must be defined (d=6), but additional bands or properties can be included too (d > 6).

In the following we do this such that for each point we have `x,y,z,r,g,b,i`, where `i` is the scalar field we loaded from our point cloud (see above).

In [6]:
# combine into a single array layout for exporting
xyzrgbattr = np.hstack([xyz, rgb, attr])
print(xyzrgbattr.shape) # everything is concatenated now into a flat array

(16224567, 7)


In [7]:
xyzrgbattr[:,8:] = xyzrgbattr[:,8:] / 255 # convert to 0-1 range

We also need to define which visualisation options we want available for this point cloud in our virtual field trip. We do this by creating a `style` dictionary that lets us define different true- and false-colour composites to view, and also define a colour-ramp for scalar attribute(s).

In [8]:
vmn = np.percentile( xyz-np.mean(xyz,axis=0)[None,:], 2, axis=0) # value used as black for our false colour mapping
vmx = np.percentile( xyz-np.mean(xyz,axis=0)[None,:], 98, axis=0) # value used as white for our false colour mapping
stylesheet = { 'rgb':{'color':{'R':(3,0,1), # dimensions of our data array to map to "red", vmin, vmax
                         'G':(4,0,1),'B':(5,0,1)} }, # other colors (to give true-colour visualisation) 
               'carbonate':{'color':{'R':(8,0,1), # example false colour visualisation
                                'G':(9,0,1),'B':(10,0,1)}},
               'clay':{'color':{'R':(11,0,1), # example false colour visualisation
                                'G':(12,0,1),'B':(13,0,1)}},
            }

vmn = np.percentile( xyz[:,2]-np.mean(xyz[:,2]), 2, axis=0) # min value of colour ramp
vmx = np.percentile( xyz[:,2]-np.mean(xyz[:,2]), 98, axis=0) # max value of colour ramp
stylesheet['elev'] = {'color':(2, # index of attribute to colour map
                             {'scale':'viridis', 'limits':(vmn,vmx,255)}) } # colour map properties

vmn = np.percentile( attr[:,0], 2, axis=0) # min value of colour ramp
vmx = np.percentile( attr[:,0], 98, axis=0) # max value of colour ramp
stylesheet['illu'] = {'color':(6, {'scale':['#ca0020','#f4a582','#f7f7f7','#92c5de','#0571b0'], # use custom colours!
                                   'limits':(vmn,vmx,16)})
                      } # and use e.g., quantile limits instead of equidistant (e) ones.

groups = {
        "dolomite": {
            "blend": 0.3,
            "color": [ 1,  1, 0 ],
            "iq": [ 7, "=", 0 ]
        },
        "marl": {
            "blend": 0.3,
            "color": [ 1, 1, 0
            ],
            "iq": [ 7, "=", 2
            ]
        },
        "scree": {
            "mask": [ 7, "=", 1 ]
        }
    }

### Exporting to zarr

Now we have everything needed to convert our points into zarr format. This is a directory structure that allows the front-end (our web-based virtual fieldtrip viewer) to stream points more efficiently. 

In [9]:
# create our new zarr directory here
out_path = '../sandbox/test.zarr'

In [10]:
exportZA( xyzrgbattr, # our array to export
          out_path, # the path to save this [for uploading to a server later]
          chunk_size=200000, # number of points in each patch streamed to the viewer
          resolution=0.1, # downsample our point cloud to this resolution (important to keep file size low)
          stylesheet=stylesheet,
          groups=groups) # groups define overlays (highlights) and masks (hide some points)
